In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/hc-rl-thompson-sampling/sepsis/custom_sepsis/src/custom_sepsis

In [ ]:
!pip install stable_baselines3

In [ ]:
!pip install dill

In [1]:
import sys
import os
# sys.path.append(os.path.abspath("/content/drive/My Drive/hc-rl-thompson-sampling/sepsis/custom_sepsis/"))

import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
priors = np.full((n_states, n_actions, n_states), 5 / (n_states * n_actions * n_states))


In [3]:
ts = DirThompsonSampling(create_model(Simplification.NONE,priors), {},{}, {},{}, f"None-small-prior-{1}", {})
policy = random_policy()
rewards = {}


In [ ]:
# history
V = np.zeros(n_states)
for ep_ix in range(50000):
    if ep_ix == 0 or (ep_ix < 100 and np.log2(ep_ix) % 1 == 0) or (ep_ix % 100 == 0): 
        state_counts = ts.model.get_state_counts()
        policy, V = matrix_value_iteration(V, ts.model.transition_model())
        rewards[ep_ix] = evaluate_policy(policy, 100000)
        ts.add_data(ep_ix, rewards, policy, state_counts)
        ts.save_json()
        print(f"Episode {ep_ix}, done")
    episode = run_episode(policy)
    ts.model.update_state_counts(episode)
        